# C-RNN-GAN
http://mogren.one/publications/2016/c-rnn-gan/mogren2016crnngan.pdf

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [29]:
class Generator(nn.Module):
    def __init__(self, features, hidden_size):
        super(Generator, self).__init__()
        
        self.hidden_size = hidden_size
        self.features = features
        
        self.fc1 = nn.Linear(in_features=(features*2), out_features=hidden_size)
        self.lstm1 = nn.LSTMCell(input_size=hidden_size, hidden_size=hidden_size)
        self.dropout = nn.Dropout(p=0.6)
        self.lstm2 = nn.LSTMCell(input_size=hidden_size, hidden_size=hidden_size)
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=features)
        
    def forward(self, z, states):
        z = z.to(device)
        batch_size, seq_len, num_feats = z.shape
        z = torch.split(z, 1, dim=1)
        z = [z_step.squeeze(dim=1) for z_step in z]
        
        prev_gen = torch.empty([batch_size, num_feats]).uniform_()
        prev_gen = prev_gen.to(device)
        
        state1, state2 = states
        gen_feats = []
        for z_step in z:
            concat_in = torch.cat((z_step, prev_gen), dim=-1)
            out = F.relu(self.fc1(concat_in))
            h1, c1 = self.lstm1(out, state1)
            h1 = self.dropout(h1)
            h2, c2 = self.lstm2(h1, state2)
            prev_gen = self.fc2(h2)
            gen_feats.append(prev_gen)
            state1 = (h1, c1)
            state2 = (h2, c2)
        
        # seq_len * (batch_size * num_feats) -> (batch_size * seq_len * num_feats)
        gen_feats = torch.stack(gen_feats, dim=1)
        
        states = (state1, state2)
        return gen_feats, states

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        hidden = ( (weight.new(batch_size, self.hidden_size).zero_().to(device),
                   weight.new(batch_size, self.hidden_size).zero_().to(device)),
                   (weight.new(batch_size, self.hidden_size).zero_().to(device),
                   weight.new(batch_size, self.hidden_size).zero_().to(device)) )

        return hidden

In [36]:
gmodel = Generator(13, 100).to(device)

In [37]:
BATCH_SIZE = 1
g_states = gmodel.init_hidden(BATCH_SIZE)

In [38]:
g_states.device()

AttributeError: 'tuple' object has no attribute 'device'

In [39]:
input_seq = torch.arange(13. * 4.).reshape(1, 4, 13).to(device)

In [41]:
gmodel.eval()
gmodel(input_seq, g_states)

(tensor([[[-0.0166,  0.0489, -0.0056,  0.0335, -0.1198,  0.0685, -0.0229,
           -0.0305,  0.0163,  0.0851,  0.0111, -0.0066, -0.0310],
          [-0.0060,  0.0401,  0.0047,  0.0394, -0.1385,  0.0625, -0.0261,
           -0.0521,  0.0299,  0.0844,  0.0046, -0.0241, -0.0734],
          [-0.0021,  0.0252,  0.0175,  0.0513, -0.1370,  0.0592, -0.0282,
           -0.0730,  0.0455,  0.0853,  0.0048, -0.0292, -0.0995],
          [-0.0037,  0.0073,  0.0270,  0.0623, -0.1307,  0.0552, -0.0266,
           -0.0876,  0.0602,  0.0841,  0.0120, -0.0269, -0.1118]]],
        device='cuda:0', grad_fn=<StackBackward>),
 ((tensor([[-8.4788e-03,  2.1228e-04,  3.9706e-02,  6.0995e-01,  7.6642e-01,
            -7.5935e-01,  2.1733e-01, -1.6564e-01,  7.1967e-01, -7.9875e-02,
            -1.8251e-06, -1.5560e-02, -2.5932e-03, -1.6780e-09,  5.9981e-08,
            -6.5191e-01,  8.6587e-06, -1.9787e-05, -3.7394e-11, -4.6595e-02,
             6.8628e-06, -9.5037e-01,  7.3292e-02,  5.5596e-10, -4.9211e-05,
  